In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import math
import sympy as sp

In [7]:

class ConcreteBeam:
    def __init__(self, b, d, fc, fy, Ast_bars=None, Ast_diam=None, Asc_bars=None, Asc_diam=None, d_prime=None):
        self.b = b  # Width of the beam (mm)
        self.d = d  # Effective depth of the beam (mm)
        self.fc = fc  # Compressive strength of concrete (MPa)
        self.fy = fy  # Yield strength of steel (MPa)
        self.Ast = self.calculate_area(Ast_bars, Ast_diam) if Ast_bars and Ast_diam else None  # Area of tensile steel (mm^2)
        self.Asc = self.calculate_area(Asc_bars, Asc_diam) if Asc_bars and Asc_diam else None  # Area of compressive steel (mm^2)
        self.d_prime = d_prime  # Distance from the compression face to the centroid of compression reinforcement (mm)
        self.beta = self.beam_material_properties()

    def calculate_area(self, num_bars, diameter):
        """Calculate the area of steel reinforcement based on the number of bars and their diameter."""
        return num_bars * (math.pi / 4) * (diameter ** 2)

    def beam_material_properties(self):
        # Determine beta based on fc
        if self.fc >= 17 and self.fc <= 28:
            beta = 0.85
        elif self.fc > 28 and self.fc < 55:
            beta = 0.85 - 0.05 * (self.fc - 28) / 7
        elif self.fc >= 55:
            beta = 0.65
        else:
            raise ValueError("fc must be greater than or equal to 17 MPa")
        return beta

    def analyze(self):
        if self.Asc is None or self.Asc == 0:
            return self.analyze_singly_reinforced()
        else:
            return self.analyze_doubly_reinforced()

    def analyze_singly_reinforced(self):
        # Singly reinforced beam analysis
        a = (self.Ast * self.fy) / (0.85 * self.fc * self.b)
        c = a / self.beta
        fs = 600 * ((self.d - c) / c)
        
        # Shift solve for new c
        c_new = sp.symbols('c_new')
        equation = sp.Eq(0.85 * self.fc * self.beta * c_new * self.b, self.Ast * (600 * (self.d - c_new) / c_new))
        c0 = sp.solve(equation, c_new)
        
        # Filter solutions to find valid c0_value
        c0_value = None
        for sol in c0:
            sol_value = sol.evalf()
            if sol_value > 0 and sol_value < self.d:
                c0_value = sol_value
                break
        if c0_value is None:
            raise ValueError("No valid solution for c found.")
        fs_new = 600 * ((self.d - c0_value) / c0_value)
        
        if fs > self.fy: 
            Mn = self.Ast * self.fy * (self.d - a / 2) / 1e6  # Steel Yields
            print(f"c: {c:.4f}")
            print(f"a: {c*self.beta:.4f}")
        else:
            Mn = self.Ast * 600 * ((self.d - c0_value) / c0_value) * (self.d - (c0_value * self.beta) / 2) / 1e6  # SDNY
            print(f"c0: {c0_value:.4f}")
            print(f"a: {c0_value*self.beta:.4f}")
            
        phi = self.failure_type_singly(fs, fs_new)
        Mu = phi * Mn  # Ultimate moment capacity
        
        return Mn, Mu, phi
        
    def failure_type_singly(self, fs, fs_new):
        # Determine the phi factor based on failure types
        if fs > self.fy:
            if self.fy <= fs <= 1000:
                phi = 0.65 + (250 / 3) * ((fs / 200000) - 0.002)
            elif fs > 1000:
                phi = 0.9
            else:
                phi = 0.65
        else:
            if self.fy <= fs_new <= 1000:
                phi = 0.65 + (250 / 3) * ((fs_new / 200000) - 0.002)
            elif fs_new > 1000:
                phi = 0.9
            else:
                phi = 0.65
        return phi
        
    def analyze_doubly_reinforced(self):
        # Initial Condition fs: yields & f's: yields
        a_doubly = (self.Ast * self.fy - (self.Asc * self.fy - 0.85*self.Asc*self.fc)) / (0.85 * self.fc * self.b)
        c_doubly = a_doubly / self.beta
        fs_doubly = 600 * (self.d - c_doubly) / c_doubly
        fs_prime = 600 * (c_doubly - self.d_prime) / c_doubly

        # Second Condition fs: yields & f's: DNY
        c1 = sp.symbols('c1')
        equation1 = sp.Eq(0.85 * self.fc * self.beta * c1 * self.b + self.Asc * 600 * ((c1 - self.d_prime) / c1)- 0.85*self.Asc*self.fc, self.Ast * self.fy)
        newc1 = sp.solve(equation1, c1)

        newc1_value = None
        for sol in newc1:
            sol_value = sol.evalf()
            if sol_value > 0 and sol_value < self.d:
                newc1_value = sol_value
                break
        if newc1_value is None:
            raise ValueError("No valid solution for c found.")

        fs_doubly_1 = 600 * (self.d - newc1_value) / newc1_value
        fs_prime_1 = 600 * (newc1_value - self.d_prime) / newc1_value

        # Third Condition fs: DNY & f's: yields
        c2 = sp.symbols('c2')
        equation2 = sp.Eq(0.85 * self.fc * self.beta * c2 * self.b + (self.Asc * self.fy - 0.85*self.Asc*self.fc), self.Ast * 600 * (self.d - c2) / c2)
        newc2 = sp.solve(equation2, c2)

        newc2_value = None
        for sol in newc2:
            sol_value = sol.evalf()
            if sol_value > 0 and sol_value < self.d:
                newc2_value = sol_value
                break
        if newc2_value is None:
            raise ValueError("No valid solution for c found.")

        fs_doubly_2 = 600 * (self.d - newc2_value) / newc2_value
        fs_prime_2 = 600 * (newc2_value - self.d_prime) / newc2_value

        # Fourth Condition fs: DNY & f's: DNY
        c3 = sp.symbols('c3')
        equation3 = sp.Eq(0.85 * self.fc * self.beta * c3 * self.b + (self.Asc * 600 * ((c3 - self.d_prime) / c3)-0.85*self.Asc*self.fc), self.Ast * 600 * (self.d - c3) / c3)
        newc3 = sp.solve(equation3, c3)

        newc3_value = None
        for sol in newc3:
            sol_value = sol.evalf()
            if sol_value > 0 and sol_value < self.d:
                newc3_value = sol_value
                break
        if newc3_value is None:
            raise ValueError("No valid solution for c found.")

        fs_doubly_3 = 600 * (self.d - newc3_value) / newc3_value
        fs_prime_3 = 600 * (newc3_value - self.d_prime) / newc3_value

        # Logic for computing Mn based on conditions
        if fs_doubly > self.fy and fs_prime > self.fy:
            Mn = (0.85 * self.fc * a_doubly * self.b * (self.d - a_doubly / 2) + (self.Asc * self.fy-0.85*self.Asc*self.fc) * (self.d - self.d_prime)) / 1e6
            print(f"Tension Yields & Compression Yields")
            print(f"c: {c_doubly:.4f}")
            print(f"a: {a_doubly:.4f}")
            phi = self.failure_type_doubly(fs_doubly, fs_prime)
        elif fs_doubly_1 > self.fy and fs_prime_1 < self.fy:
            Mn = (0.85 * self.fc * self.beta * newc1_value * self.b * (self.d - self.beta * newc1_value / 2) + (self.Asc * 600 * ((newc1_value - self.d_prime) / newc1_value)-0.85*self.Asc*self.fc) * (self.d - self.d_prime)) / 1e6
            print(f"Tension Yields & Compression DNY")
            print(f"c: {newc1_value:.4f}")
            print(f"a: {newc1_value * self.beta:.4f}")
            phi = self.failure_type_doubly(fs_doubly_1, fs_prime_1)
        elif fs_doubly_2 < self.fy and fs_prime_2 > self.fy:
            Mn = (0.85 * self.fc * self.beta * newc2_value * self.b * (self.d - self.beta * newc2_value / 2) + (self.Asc * self.fy - 0.85*self.Asc*self.fc) * (self.d - self.d_prime)) / 1e6
            print(f"Tension DNY & Compression Yields")
            print(f"c: {newc2_value:.4f}")
            print(f"a: {newc2_value * self.beta:.4f}")
            phi = self.failure_type_doubly(fs_doubly_2, fs_prime_2)
        elif fs_doubly_3 < self.fy and fs_prime_3 < self.fy:
            Mn = (0.85 * self.fc * self.beta * newc3_value * self.b * (self.d - self.beta * newc3_value / 2) + (self.Asc * 600 * ((newc3_value - self.d_prime) / newc3_value)-0.85*self.Asc*self.fc) * (self.d - self.d_prime)) / 1e6
            print(f"Tension DNY & Compression DNY")
            print(f"c: {newc3_value:.4f}")
            print(f"a: {newc3_value * self.beta:.4f}")
            phi = self.failure_type_doubly(fs_doubly_3, fs_prime_3)

        Mu = phi * Mn  # Ultimate moment capacity
        
        return Mn, Mu, phi

    def failure_type_doubly(self, fs_doubly, fs_prime):
        # Determine the phi factor based on failure types for doubly reinforced beams
        if fs_doubly > self.fy and fs_prime > self.fy:
            if self.fy <= fs_doubly <= 1000:
                phi = 0.65 + (250 / 3) * ((fs_doubly / 200000) - 0.002)
            elif fs_doubly > 1000:
                phi = 0.9
            else:
                phi = 0.65
        elif fs_doubly > self.fy and fs_prime < self.fy:
            if self.fy <= fs_doubly <= 1000:
                phi = 0.65 + (250 / 3) * ((fs_doubly / 200000) - 0.002)
            elif fs_doubly > 1000:
                phi = 0.9
            else:
                phi = 0.65
        elif fs_doubly < self.fy and fs_prime > self.fy:
            if self.fy <= fs_prime <= 1000:
                phi = 0.65 + (250 / 3) * ((fs_prime / 200000) - 0.002)
            elif fs_prime > 1000:
                phi = 0.9
            else:
                phi = 0.65
        elif fs_doubly < self.fy and fs_prime < self.fy:
            if self.fy <= fs_prime <= 1000:
                phi = 0.65 + (250 / 3) * ((fs_prime / 200000) - 0.002)
            elif fs_prime > 1000:
                phi = 0.9
            else:
                phi = 0.65
        else:
            phi = 0.65
        return phi
        
    def analyze(self):
        if self.Asc is None or self.Asc == 0:
            return self.analyze_singly_reinforced()
        else:
            return self.analyze_doubly_reinforced()

# User Input Section
b = float(input("Enter the beam width (mm): "))
d = float(input("Enter the effective depth (d) (mm): "))
fc = float(input("Enter the concrete compressive strength (fc) (MPa): "))
fy = float(input("Enter the yield strength of steel (fy) (MPa): "))

Ast_bars = int(input("Enter the number of tensile bars: "))
Ast_diam = float(input("Enter the diameter of tensile bars (mm): "))

Asc_bars = int(input("Enter the number of compressive bars (Enter 0 if none): "))
Asc_diam = float(input("Enter the diameter of compressive bars (mm, Enter 0 if none): "))

d_prime = float(input("Enter the distance from the compression face to the centroid of compression reinforcement (d') (mm): "))

# Create the ConcreteBeam object
beam = ConcreteBeam(b, d, fc, fy, Ast_bars, Ast_diam, Asc_bars, Asc_diam, d_prime)

# Analyze the beam
Mn, Mu, phi = beam.analyze()

# Output the results
if Asc_bars == 0:
    print(f"Singly Reinforced Beam Moment of Resistance (Mn): {Mn:.4f} kNm")
    print(f"Singly Reinforced Beam Ultimate Moment Capacity (Mu): {Mu:.4f} kNm")
else:
    print(f"Doubly Reinforced Beam Moment of Resistance (Mn): {Mn:.4f} kNm")
    print(f"Doubly Reinforced Beam Ultimate Moment Capacity (Mu): {Mu:.4f} kNm")
print(f"Phi Factor: {phi:.4f}")



Enter the beam width (mm):  350
Enter the effective depth (d) (mm):  624
Enter the concrete compressive strength (fc) (MPa):  20
Enter the yield strength of steel (fy) (MPa):  420
Enter the number of tensile bars:  4
Enter the diameter of tensile bars (mm):  32
Enter the number of compressive bars (Enter 0 if none):  2
Enter the diameter of compressive bars (mm, Enter 0 if none):  20
Enter the distance from the compression face to the centroid of compression reinforcement (d') (mm):  60


Tension Yields & Compression Yields
c: 217.0882
a: 184.5250
Doubly Reinforced Beam Moment of Resistance (Mn): 726.6190 kNm
Doubly Reinforced Beam Ultimate Moment Capacity (Mu): 653.9571 kNm
Phi Factor: 0.9000
